<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 notebook 是為了選取剪輯 news-digest 的部分新聞分類，以利後續轉貼
執行步驟如下：
1. 按下 Install 下那個 code cell 前面的 ► 按鈕來安裝 clip 模組.
2. 根據需要，填表執行後面的 Forms 或 Widgets UI

### Install 

In [31]:
!curl -o clip.py https://raw.githubusercontent.com/YorkJong/news-digest/main/src/clip.py
from clip import *

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4967  100  4967    0     0  24835      0 --:--:-- --:--:-- --:--:-- 24835


### Select Categories with the Colab Forms

In [36]:
#@title Single Selection { run: "auto" }
category = "Taiwan" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = True #@param {type:"boolean"}

fn = get_latest_fn("journals")
content = get_file(fn)
lines = get_lines_of_categories([category], content, with_hashtags, with_headers)
print('\n'.join(lines))

# Copy text to clipboard in Python using pandas module
#import pandas as pd
#df=pd.DataFrame(['Text to copy'])
#df.to_clipboard(index=False, header=False)

### Taiwan
- [跟上特斯拉腳步 和碩增資墨西哥廠逾15億元](https://ec.ltn.com.tw/article/breakingnews/4246868)
- [momo 併 MyBook 電子書，總座證實將推全新閱讀器](https://technews.tw/2023/03/21/president-jeff-ku-looking-forward-to-momo-this-year/)
- [光一台EUV，每天就耗掉3萬度電！工業用電大漲17％，會衝擊台積電嗎？](https://www.bnext.com.tw/article/74507/tssmc-euv-pg-cost-impact)
- [童子賢「吃誠實豆沙包」：外商來台設資料中心，只是在用台灣低廉電價⋯](https://www.bnext.com.tw/article/74504/tongtzuhs-tw-elec-price-cheap)
- [經濟部加速軍用商規無人機國產化 與7家廠商簽約](https://m.cnyes.com/news/id/5117623)
- [【國內新訊】氫燃料動力在台首賣？Toyota宣布將專案引進Mirai電動車！ (第3頁)](https://m.mobile01.com/topicdetail.php?f=264&p=3&t=6758523)
- [緯創搭上AI熱潮，砸62億元進軍數位內容！硬體起家，為何頻頻跨界做軟體？](https://www.bnext.com.tw/article/74493/wistron-bot)
-


In [ ]:
#@title Multiple Selections { run: "auto" }
first_category = "Tesla & SpaceX; Vehicle" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
last_category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = True #@param {type:"boolean"}

fn = get_latest_fn("journals")
content = get_file(fn)
categories = get_categories(content)
categories = get_sublist(categories, first_category, last_category)
lines = get_lines_of_categories(categories, content, with_hashtags, with_headers)
print('\n'.join(lines))


### Select multiple categories with Jupyter Widgets

In [38]:
import ipywidgets as widgets

fn = get_latest_fn("journals")
content = get_file(fn)
options = get_categories(content)

categories = widgets.SelectMultiple(
    options=options,
    value=[options[0]],
    rows=len(options),
    description='Categories',
    disabled=False
)

with_hashtags = widgets.Checkbox(
    value=True,
    description='with hashtags',
    disabled=False
)

with_headers = widgets.Checkbox(
    value=True,
    description='with headers',
    disabled=False
)

flags = widgets.VBox([with_hashtags, with_headers])
widgets.HBox([categories, flags])

In [ ]:
lines = get_lines_of_categories(categories.value, content, with_hashtags.value, with_headers.value)
print('\n'.join(lines))